In [1]:
import pandas as pd

In [18]:
with open('uniprot_ids', 'r') as file:
    uniprot_ids = file.read()
    uniprot_ids = uniprot_ids.split('\n')
    print(uniprot_ids)

['P10321', 'P04439', 'P01899', 'P17693', 'P13747']


In [25]:
import requests

def getDataFromKegg(operation, argument):
    url = f"https://rest.kegg.jp/{operation}/{argument}"

    resp = requests.get(
        url
    )

    if resp.ok:
        return resp.text

for uniprot_id in uniprot_ids:
    print(getDataFromKegg('conv', f'genes/uniprot:{uniprot_id}').split()[1])

hsa:3107
hsa:3105
mmu:14964
hsa:3135
hsa:3133


In [32]:
def getAaSeq(uniprot_id):
    kegg_id_data = getDataFromKegg(
        "conv", f"genes/uniprot:{uniprot_id}"
    )

    kegg_id = kegg_id_data.strip().split("\t")[1]

    gene = getDataFromKegg(
        "get", f"{kegg_id}/aaseq"
    )

    #gene = "".join(gene.split("\n")[1:])

    return gene

with open('fastaFromKegg', 'w') as file:
    for uniprot_id in uniprot_ids:
        file.write(getAaSeq(uniprot_id))



In [36]:
import time

def runClustal(sequecnes_in_fasta):
    resp = requests.post(
        'https://www.ebi.ac.uk/Tools/services/rest/clustalo/run',
        data={
            "email": "marcel.thiel@ug.edu.pl",
            "sequence": sequecnes_in_fasta,
            "outfmt": "fa"
        }
    )
    jobid = resp.text

    return jobid


def getStatus(jobId):
    url = f"https://www.ebi.ac.uk/Tools/services/rest/clustalo/status/{jobId}"
    
    resp = requests.get(
        url
    )

    return resp.text


def getResults(jobId):
    url = f"https://www.ebi.ac.uk/Tools/services/rest/clustalo/result/{jobId}/fa"

    resp = requests.get(
        url
    )

    return resp.text

with open('fastaFromKegg', 'r') as file:
    sequencesFromFasta = file.read()

jobid = runClustal(sequencesFromFasta)

print(f"JOB ID IS: {jobid}")

max_attempt = 20

for attept in range(max_attempt):
    status = getStatus(jobid)

    if status == "FINISHED":
        results = getResults(jobid)
        print(results)

        break

    time.sleep(1)

def countGaps(result: str):
    resultsInLines = result.split('\n')
    gapsCount = 0
    for cos in resultsInLines:
        if cos.startswith('>'):
            pass
        else:
            gapsCount += cos.count('-')
    return gapsCount

print(countGaps(results))    


JOB ID IS: clustalo-R20240222-180140-0823-19763603-p1m
>3107 K06751 MHC class I antigen | (RefSeq) HLA-C, D6S204, HLA-JY3, HLAC, HLC-C, MHC, PSORS1; major histocompatibility complex, class
MRVMAPRALLLLLSGGLALTETWACSHSMRYFDTAVSRPGRGEPRFISVGYVDDTQFVRF
DSDAASPRGEPRAPWVEQEGPEYWDRETQKYKRQAQADRVSLRNLRGYYNQSEDGSHTLQ
RMSGCDLGPDGRLLRGYDQSAYDGKDYIALNEDLRSWTAADTAAQITQRKLEAARAAEQL
RAYLEGTCVEWLRRYLENGKETLQRAEPPKTHVTHHPLSDHEATLRCWALGFYPAEITLT
WQRDGEDQTQDTELVETRPAGDGTFQKWAAVVVPSGQEQRYTCHMQHEGLQEPLTLSWEP
SSQPTIPIMGIVAGLAVLVVLAVLGAVVTAMMCRRKSSGGKGGSCSQAACSNSAQGSDES
LITCKA
>3105 K06751 MHC class I antigen | (RefSeq) HLA-A, HLAA; major histocompatibility complex, class I, A (A)
MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRF
DSDAASQKMEPRAPWIEQEGPEYWDQETRNMKAHSQTDRANLGTLRGYYNQSEDGSHTIQ
IMYGCDVGPDGRFLRGYRQDAYDGKDYIALNEDLRSWTAADMAAQITKRKWEAVHAAEQR
RVYLEGRCVDGLRRYLENGKETLQRTDPPKTHMTHHPISDHEATLRCWALGFYPAEITLT
WQRDGEDQTQDTELVETRPAGDGTFQKWAAVVVPSGEEQRYTCHVQHEGLPKPLTLRWEL
SSQPTIPIVGIIAGLVLLGA-VITGAV